In [ ]:
from function_library import *
from reformulation_library import *

In [ ]:
def shortest_path_replication(grid_dim,
    n_train, n_holdout, n_test,
    p_features, polykernel_degree, polykernel_noise_half_width,
    num_lambda = 10, lambda_max = None, lambda_min_ratio = 0.0001, regularization = 'ridge',
    different_validation_losses = False,
    include_rf = True):

    different_validation_losses = False
    n_holdout = 1000
    n_test = 1000
    grid_dim = 5
    p_features = 5
    n_train = 1000
    polykernel_degree = 1
    polykernel_noise_half_width = 0
    d_feasibleregion = 2 * p_features * (p_features - 1)
    sources, destinations = convert_grid_to_list(grid_dim, grid_dim)
    sp_graph = shortest_path_graph(sources = sources, destinations = destinations,
            start_node = 1, end_node = grid_dim^2, acyclic = True)
    B_true = np.array([[bernoulli(0.5) for k in range(p_features)] for e in range(d_feasibleregion)])
    X_train, c_train = generate_poly_kernel_data_simple(B_true, n_train, polykernel_degree, polykernel_noise_half_width)
    X_validation, c_validation = generate_poly_kernel_data_simple(B_true, n_holdout, polykernel_degree, polykernel_noise_half_width)
    X_test, c_test = generate_poly_kernel_data_simple(B_true, n_test, polykernel_degree, polykernel_noise_half_width)
    # Add intercept in the first row of X
    # an intercept term is a constant term that is added to a linear model to 
    # account for the mean or baseline level of the target variable.
    # X_train = np.vstack((np.ones(n_train), X_train))
    # X_validation = np.vstack((np.ones(n_holdout), X_validation))
    # X_test = np.vstack((np.ones(n_test), X_test))

    # Solve the shortest path problem
    G = define_graph(grid_dim, showflag=False)
    solver = Shortest_path_solver(G)
    z_train, w_train = batch_solve(solver, c_train)
    z_validation, w_validation = batch_solve(solver, c_validation)
    z_test, w_test = batch_solve(solver, c_test)

    c_ham_train = np.ones((d_feasibleregion, n_train)) - w_train
    c_ham_validation = np.ones((d_feasibleregion, n_holdout)) - w_validation
    c_ham_test = np.ones((d_feasibleregion, n_test)) - w_test

    # Put train + validation together
    X_both = np.hstack((X_train, X_validation))
    c_both = np.hstack((c_train, c_validation))
    c_ham_both = np.hstack((c_ham_train, c_ham_validation))
    train_index = np.arange(n_train)
    validation_index = np.arange(n_train, n_train + n_holdout)

    # Set validation losses
    if different_validation_losses:
        spo_plus_val_loss = 'spo_loss'
        ls_val_loss = 'least_squares_loss'
        absolute_val_loss = 'absolute_loss'
    else:
        spo_plus_val_loss = 'spo_loss'
        ls_val_loss = 'spo_loss'
        absolute_val_loss = 'spo_loss'
    ### Algorithms ###

    # SPO+
    best_B_SPOplus, best_lambda_SPOplus = validation_set_alg(X_both, c_both, solver; sp_graph = sp_graph,
        train_ind = train_ind, validation_ind = validation_ind,
        val_alg_parms = val_parms(algorithm_type = :sp_spo_plus_reform, validation_loss = spo_plus_val_loss),
        path_alg_parms = reformulation_path_parms(num_lambda = num_lambda, lambda_max = lambda_max, 
                                                regularization = regularization, 
                                                gurobiEnv = gurobiEnvReform, 
                                                lambda_min_ratio = lambda_min_ratio, algorithm_type = :SPO_plus))
    # Least squares
    best_B_leastSquares, best_lambda_leastSquares = validation_set_alg(X_both, c_both, solver; sp_graph = sp_graph,
        train_ind = train_ind, validation_ind = validation_ind,
        val_alg_parms = val_parms(algorithm_type = :ls_jump, validation_loss = ls_val_loss),
        path_alg_parms = reformulation_path_parms(num_lambda = num_lambda, lambda_max = lambda_max, 
                                                regularization = regularization,
                                                gurobiEnv = gurobiEnvReform, lambda_min_ratio = lambda_min_ratio, 
                                                algorithm_type = :leastSquares)) 
    # Absolute
    best_B_Absolute, best_lambda_Absolute = validation_set_alg(X_both, c_both, solver; sp_graph = sp_graph,
        train_ind = train_ind, validation_ind = validation_ind,
        val_alg_parms = val_parms(algorithm_type = 'ls_jump', validation_loss = 'absolute_val_loss'),
        path_alg_parms = reformulation_path_parms(num_lambda = num_lambda, lambda_max = lambda_max, 
                                                regularization = regularization,
                                                po_loss_function = :absolute, gurobiEnv = gurobiEnvReform, 
                                                lambda_min_ratio = lambda_min_ratio, algorithm_type = :Absolute))
    # RF
    if include_rf:
        rf_mods = train_random_forests_po(X_train, c_train, rf_alg_parms = rf_parms())


    ### Populate final results ###
    final_results = replication_results()

    final_results.SPOplus_spoloss_test = spo_loss(best_B_SPOplus, X_test, c_test, solver)
    final_results.LS_spoloss_test = spo_loss(best_B_leastSquares, X_test, c_test, solver)
    final_results.Absolute_spoloss_test = spo_loss(best_B_Absolute, X_test, c_test, sp_oracle)

    if include_rf:
        rf_preds_test = predict_random_forests_po(rf_mods, X_test)
        final_results.RF_spoloss_test = spo_loss(Matrix(1.0I, d_feasibleregion, d_feasibleregion), rf_preds_test, c_test, sp_oracle)
    else:
        final_results.RF_spoloss_test = None

    final_results.zstar_avg_test = np.mean(z_test)

    return final_results


In [ ]:
import pandas as pd


def shortest_path_multiple_replications(rng_seed, num_trials, grid_dim, n_train_vec, n_test, p_features, polykernel_degree_vec, polykernel_noise_half_width_vec, num_lambda=10, lambda_max=None, lambda_min_ratio=0.0001, holdout_percent=0.25, regularization='ridge', different_validation_losses=False, include_rf=True):
    np.random.seed(rng_seed)
    big_results_df = make_blank_complete_df()

    for n_train in n_train_vec:
        for polykernel_degree in polykernel_degree_vec:
            for polykernel_noise_half_width in polykernel_noise_half_width_vec:
                print(f"Moving on to n_train = {n_train}, polykernel_degree = {polykernel_degree}, polykernel_noise_half_width = {polykernel_noise_half_width}")
                for trial in range(num_trials):
                    print(f"Current trial is {trial}")
                    n_holdout = round(holdout_percent*n_train)

                    current_results = shortest_path_replication(grid_dim,
                        n_train, n_holdout, n_test,
                        p_features, polykernel_degree, polykernel_noise_half_width,
                        num_lambda=num_lambda, lambda_max=lambda_max, lambda_min_ratio=lambda_min_ratio,
                        regularization=regularization,
                        different_validation_losses=different_validation_losses,
                        include_rf=include_rf)

                    current_df_row = build_complete_row(grid_dim,
                        n_train, n_holdout, n_test,
                        p_features, polykernel_degree, polykernel_noise_half_width, current_results)

                    big_results_df = pd.concat([big_results_df, current_df_row])

    return big_results_df
